## Check cconv Implementation

In [1]:
def enable_parent_folder_import():
    import sys, os
    sys.path.insert(1, os.path.join(sys.path[0], '..'))

enable_parent_folder_import()

In [2]:
from importlib import reload
import datasets.density_data_module
from datasets.density_data_module import *
reload(datasets.density_data_module); from datasets.density_data_module import *

density_data = DensityDataModule(
    data_dir = '../datasets/data/dpi_dam_break/train',
    batch_size = 10,
    data_split = (0.7, 0.15, 0.15),
    num_workers = 0, # Note that cuda only allows 0 workers.
    shuffle = False,
    cache = True, # Load dataset into memory
    device = 'cuda',
)
density_data.setup("fit")
train_loader = density_data.train_dataloader()
train_iter = iter(train_loader)
batch = next(train_iter)
sample = batch[0]

Setting up data module for stage  fit
Loading dataset into memory and applying transform_once... (Device: cuda)


  0%|          | 0/131 [00:00<?, ?it/s]

100%|██████████| 131/131 [00:16<00:00,  7.84it/s]

Done loading dataset into memory


In [35]:
torch.arange(5)

tensor([0, 1, 2, 3, 4])

In [41]:
import models.cconv
from models.cconv import *
reload(models.cconv); from models.cconv import *

hparams = {}
hparams['batch_size'] = 10
hparams['lr'] = 1e-3

model = CConvModel(hparams)
model.to('cuda')
model(sample)

tensor([[-0.0042,  0.0038, -0.0072],
        [-0.0016,  0.0021, -0.0056],
        [-0.0045,  0.0033, -0.0027],
        ...,
        [-0.0180,  0.0202, -0.0106],
        [-0.0113,  0.0242, -0.0096],
        [-0.0062,  0.0283, -0.0011]], device='cuda:0', grad_fn=<MulBackward0>)